# 1. Producing the data  
In this task, we will implement Apache Kafka producers to simulate real-time data streaming. Spark is not required in this part since it’s simulating a streaming data source.  
  
1.	Your program should send one batch of order data every 5 seconds. One batch consists of a random 20-50 rows (i.e. orders) from the new_order.csv. The CSV shouldn’t be loaded to memory at once to conserve memory (i.e. Read row as needed).   
(note: you can read the file sequentially; only the number of rows is random.)  
2.	Update order_ts to the current timestamp and spread your batch out evenly for 5 seconds. Update ready_ts if necessary (when you use it in your ML model).  
a.	For example, assume you send a batch of 50 records at 2025-01-26 00:00:00 (ISO format: YYYY-MM-DD HH:MM:SS) -> (ts = 1737810000):  
-	Record 1-10: ts = 1737810000
-	Record 11-20: ts = 1737810001
-	Record 21-30: ts = 1737810002
-	….  
3.	Set the delivery_persion_id and delivery_time to null (because the order just came in, we haven’t assigned a delivery person yet).  
4.	Send your order batch to a Kafka topic with an appropriate name.  
5.	Every 5 seconds, randomly select 50 drivers from the driver dataset, send it to another Kafka topic. (note: this simplifies the application, ensuring all orders have at least one available driver.)  



In [ ]:
# Import statements
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import datetime as dt
import csv

# Configurations
hostip = "kafka"  # Change this to your Kafka broker IP
order_file = "new_order.csv"
driver_file = "driver.csv"

# Kafka Topics
order_topic = "new_orders"
driver_topic = "available_drivers"

def read_csv_stream(file_name):
    """ Generator function to read CSV file row by row. """
    with open(file_name, 'rt') as f:
        reader = csv.DictReader(f)
        for row in reader:
            yield row  # Yield one row at a time

def publish_message(producer_instance, topic_name, data):
    """ Publish message to Kafka topic. """
    try:
        producer_instance.send(topic_name, value=data)
       # print(f'Message published successfully to {topic_name}. Data: {data}')
    except Exception as ex:
        print(f'Exception in publishing message to {topic_name}.')
        print(str(ex))

def connect_kafka_producer():
    """ Connects to Kafka producer. """
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  value_serializer=lambda x: dumps(x).encode('utf-8'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

if __name__ == '__main__':
    print('Starting Kafka Producer...')

    producer = connect_kafka_producer()

    if producer is not None:
        order_stream = read_csv_stream(order_file)  # Read orders sequentially
        all_drivers = list(read_csv_stream(driver_file))  # Load all drivers once into memory

        while True:
            # --- Send Orders to Kafka ---
            num_orders = random.randint(20, 50)  # Pick a random batch size (20-50)
            order_batch = []
            
            # Pick 50 drivers (ensuring enough for all orders)
            available_drivers = random.sample(all_drivers, max(num_orders, 50))

            current_timestamp = dt.datetime.utcnow()
            base_ts = int(current_timestamp.timestamp())  # Base timestamp (epoch seconds)

            for i in range(num_orders):
                try:
                    order = next(order_stream)  # Read next order row
                except StopIteration:  # If EOF is reached, restart the generator
                    order_stream = read_csv_stream(order_file)
                    order = next(order_stream)

                # Assign a driver to this order
                assigned_driver = available_drivers[i % len(available_drivers)]  # Cycle through available drivers
                
                # Modify order fields
                order["order_ts"] = base_ts + (i // 10)  # Spread timestamps over 5 seconds
                order["ready_ts"] = order["order_ts"] + random.randint(2, 10)  # Add some prep time
                order["delivery_person_id"] = assigned_driver["driver_id"]  # Assign driver
                order["delivery_time"] = None  # Not delivered yet

                order_batch.append(order)

            # Publish batch of orders
            for order in order_batch:
                publish_message(producer, order_topic, order)

            # --- Send Drivers to Kafka ---
            for driver in available_drivers:
                publish_message(producer, driver_topic, driver)

            print(f"Published {len(order_batch)} orders (each assigned a driver) and {len(available_drivers)} drivers to Kafka.")

            # Wait for 5 seconds before the next batch
            sleep(5)

    else:
        print("Kafka producer connection failed.")


Starting Kafka Producer...
Published 37 orders (each assigned a driver) and 50 drivers to Kafka.
Published 50 orders (each assigned a driver) and 50 drivers to Kafka.
Published 48 orders (each assigned a driver) and 50 drivers to Kafka.
Published 40 orders (each assigned a driver) and 50 drivers to Kafka.
Published 23 orders (each assigned a driver) and 50 drivers to Kafka.
Published 24 orders (each assigned a driver) and 50 drivers to Kafka.
Published 27 orders (each assigned a driver) and 50 drivers to Kafka.
Published 25 orders (each assigned a driver) and 50 drivers to Kafka.
Published 40 orders (each assigned a driver) and 50 drivers to Kafka.
Published 46 orders (each assigned a driver) and 50 drivers to Kafka.
Published 41 orders (each assigned a driver) and 50 drivers to Kafka.
Published 44 orders (each assigned a driver) and 50 drivers to Kafka.
Published 50 orders (each assigned a driver) and 50 drivers to Kafka.
Published 24 orders (each assigned a driver) and 50 drivers to 

Published 50 orders (each assigned a driver) and 50 drivers to Kafka.
Published 47 orders (each assigned a driver) and 50 drivers to Kafka.
Published 27 orders (each assigned a driver) and 50 drivers to Kafka.
Published 48 orders (each assigned a driver) and 50 drivers to Kafka.
Published 35 orders (each assigned a driver) and 50 drivers to Kafka.
Published 25 orders (each assigned a driver) and 50 drivers to Kafka.
Published 26 orders (each assigned a driver) and 50 drivers to Kafka.
Published 35 orders (each assigned a driver) and 50 drivers to Kafka.
Published 37 orders (each assigned a driver) and 50 drivers to Kafka.
Published 37 orders (each assigned a driver) and 50 drivers to Kafka.
Published 35 orders (each assigned a driver) and 50 drivers to Kafka.
Published 31 orders (each assigned a driver) and 50 drivers to Kafka.
Published 27 orders (each assigned a driver) and 50 drivers to Kafka.
Published 21 orders (each assigned a driver) and 50 drivers to Kafka.
Published 35 orders 

Published 21 orders (each assigned a driver) and 50 drivers to Kafka.
Published 26 orders (each assigned a driver) and 50 drivers to Kafka.
Published 21 orders (each assigned a driver) and 50 drivers to Kafka.
Published 22 orders (each assigned a driver) and 50 drivers to Kafka.
Published 44 orders (each assigned a driver) and 50 drivers to Kafka.
Published 40 orders (each assigned a driver) and 50 drivers to Kafka.
Published 26 orders (each assigned a driver) and 50 drivers to Kafka.
Published 50 orders (each assigned a driver) and 50 drivers to Kafka.
Published 47 orders (each assigned a driver) and 50 drivers to Kafka.
Published 47 orders (each assigned a driver) and 50 drivers to Kafka.
Published 25 orders (each assigned a driver) and 50 drivers to Kafka.
Published 20 orders (each assigned a driver) and 50 drivers to Kafka.
Published 30 orders (each assigned a driver) and 50 drivers to Kafka.
Published 33 orders (each assigned a driver) and 50 drivers to Kafka.
Published 26 orders 

Published 46 orders (each assigned a driver) and 50 drivers to Kafka.
Published 47 orders (each assigned a driver) and 50 drivers to Kafka.
Published 46 orders (each assigned a driver) and 50 drivers to Kafka.
Published 49 orders (each assigned a driver) and 50 drivers to Kafka.
Published 21 orders (each assigned a driver) and 50 drivers to Kafka.
Published 30 orders (each assigned a driver) and 50 drivers to Kafka.
Published 48 orders (each assigned a driver) and 50 drivers to Kafka.
Published 29 orders (each assigned a driver) and 50 drivers to Kafka.
Published 23 orders (each assigned a driver) and 50 drivers to Kafka.
Published 48 orders (each assigned a driver) and 50 drivers to Kafka.
Published 50 orders (each assigned a driver) and 50 drivers to Kafka.
Published 50 orders (each assigned a driver) and 50 drivers to Kafka.
Published 30 orders (each assigned a driver) and 50 drivers to Kafka.
Published 25 orders (each assigned a driver) and 50 drivers to Kafka.
Published 35 orders 

Published 31 orders (each assigned a driver) and 50 drivers to Kafka.
Published 48 orders (each assigned a driver) and 50 drivers to Kafka.
Published 37 orders (each assigned a driver) and 50 drivers to Kafka.
Published 29 orders (each assigned a driver) and 50 drivers to Kafka.
Published 36 orders (each assigned a driver) and 50 drivers to Kafka.
Published 31 orders (each assigned a driver) and 50 drivers to Kafka.
Published 44 orders (each assigned a driver) and 50 drivers to Kafka.
Published 22 orders (each assigned a driver) and 50 drivers to Kafka.
Published 32 orders (each assigned a driver) and 50 drivers to Kafka.
Published 30 orders (each assigned a driver) and 50 drivers to Kafka.
Published 24 orders (each assigned a driver) and 50 drivers to Kafka.
Published 47 orders (each assigned a driver) and 50 drivers to Kafka.
Published 22 orders (each assigned a driver) and 50 drivers to Kafka.
Published 44 orders (each assigned a driver) and 50 drivers to Kafka.
Published 48 orders 

...

Published 35 orders (each assigned a driver) and 50 drivers to Kafka.
Published 37 orders (each assigned a driver) and 50 drivers to Kafka.
Published 38 orders (each assigned a driver) and 50 drivers to Kafka.
Published 43 orders (each assigned a driver) and 50 drivers to Kafka.
Published 33 orders (each assigned a driver) and 50 drivers to Kafka.
Published 24 orders (each assigned a driver) and 50 drivers to Kafka.
Published 22 orders (each assigned a driver) and 50 drivers to Kafka.
Published 26 orders (each assigned a driver) and 50 drivers to Kafka.
Published 49 orders (each assigned a driver) and 50 drivers to Kafka.
Published 28 orders (each assigned a driver) and 50 drivers to Kafka.
Published 45 orders (each assigned a driver) and 50 drivers to Kafka.
Published 25 orders (each assigned a driver) and 50 drivers to Kafka.
Published 27 orders (each assigned a driver) and 50 drivers to Kafka.
Published 33 orders (each assigned a driver) and 50 drivers to Kafka.
Published 35 orders 

Published 44 orders (each assigned a driver) and 50 drivers to Kafka.
Published 24 orders (each assigned a driver) and 50 drivers to Kafka.
Published 22 orders (each assigned a driver) and 50 drivers to Kafka.
Published 33 orders (each assigned a driver) and 50 drivers to Kafka.
Published 45 orders (each assigned a driver) and 50 drivers to Kafka.
Published 33 orders (each assigned a driver) and 50 drivers to Kafka.
Published 24 orders (each assigned a driver) and 50 drivers to Kafka.
Published 48 orders (each assigned a driver) and 50 drivers to Kafka.
Published 26 orders (each assigned a driver) and 50 drivers to Kafka.
Published 37 orders (each assigned a driver) and 50 drivers to Kafka.
Published 22 orders (each assigned a driver) and 50 drivers to Kafka.
Published 20 orders (each assigned a driver) and 50 drivers to Kafka.
Published 20 orders (each assigned a driver) and 50 drivers to Kafka.
Published 47 orders (each assigned a driver) and 50 drivers to Kafka.
Published 29 orders 

Published 23 orders (each assigned a driver) and 50 drivers to Kafka.
Published 41 orders (each assigned a driver) and 50 drivers to Kafka.
Published 29 orders (each assigned a driver) and 50 drivers to Kafka.
Published 35 orders (each assigned a driver) and 50 drivers to Kafka.
Published 50 orders (each assigned a driver) and 50 drivers to Kafka.
Published 31 orders (each assigned a driver) and 50 drivers to Kafka.
Published 26 orders (each assigned a driver) and 50 drivers to Kafka.
Published 46 orders (each assigned a driver) and 50 drivers to Kafka.
Published 35 orders (each assigned a driver) and 50 drivers to Kafka.
Published 26 orders (each assigned a driver) and 50 drivers to Kafka.
Published 24 orders (each assigned a driver) and 50 drivers to Kafka.
Published 32 orders (each assigned a driver) and 50 drivers to Kafka.
Published 36 orders (each assigned a driver) and 50 drivers to Kafka.
Published 39 orders (each assigned a driver) and 50 drivers to Kafka.
Published 40 orders 

Published 34 orders (each assigned a driver) and 50 drivers to Kafka.
Published 27 orders (each assigned a driver) and 50 drivers to Kafka.
Published 45 orders (each assigned a driver) and 50 drivers to Kafka.
Published 49 orders (each assigned a driver) and 50 drivers to Kafka.
Published 24 orders (each assigned a driver) and 50 drivers to Kafka.
Published 31 orders (each assigned a driver) and 50 drivers to Kafka.
Published 20 orders (each assigned a driver) and 50 drivers to Kafka.
Published 20 orders (each assigned a driver) and 50 drivers to Kafka.
Published 24 orders (each assigned a driver) and 50 drivers to Kafka.
Published 36 orders (each assigned a driver) and 50 drivers to Kafka.
Published 42 orders (each assigned a driver) and 50 drivers to Kafka.
Published 42 orders (each assigned a driver) and 50 drivers to Kafka.
Published 22 orders (each assigned a driver) and 50 drivers to Kafka.
Published 41 orders (each assigned a driver) and 50 drivers to Kafka.
Published 20 orders 

Published 47 orders (each assigned a driver) and 50 drivers to Kafka.
Published 22 orders (each assigned a driver) and 50 drivers to Kafka.
Published 31 orders (each assigned a driver) and 50 drivers to Kafka.
Published 45 orders (each assigned a driver) and 50 drivers to Kafka.
Published 31 orders (each assigned a driver) and 50 drivers to Kafka.
Published 30 orders (each assigned a driver) and 50 drivers to Kafka.
Published 33 orders (each assigned a driver) and 50 drivers to Kafka.
Published 50 orders (each assigned a driver) and 50 drivers to Kafka.
Published 38 orders (each assigned a driver) and 50 drivers to Kafka.
Published 27 orders (each assigned a driver) and 50 drivers to Kafka.
Published 27 orders (each assigned a driver) and 50 drivers to Kafka.
Published 48 orders (each assigned a driver) and 50 drivers to Kafka.
Published 38 orders (each assigned a driver) and 50 drivers to Kafka.
Published 42 orders (each assigned a driver) and 50 drivers to Kafka.
Published 30 orders 